# Shiny and Deployment

* App architecture
* UI, server
* Reactions, appearance
* Demos with code snippets
* Deploying Shiny apps to the web
* Deploying to Heroku

In [ ]:
library(dplyr)
library(tidyr)
library(magrittr)

## Introductory Example

In [ ]:
library(shiny)
library(ggplot2)

In [ ]:
## make data
df0 <- readRDS("small_data/inpatient_charges_2014_clean_1000.RDS")
names(df0) %<>% gsub("\\.", "", .)

df <- df0 %>% select(AverageCoveredCharges, AverageTotalPayments,
                     AverageMedicarePayments, TotalDischarges)
variables <- names(df)[1:3]
colors <- names(df)[4]

In [ ]:
# the shiny code
server <- function(input, output){
    
    output$my_output_plot <- renderPlot({

        p <- ggplot(df, aes_string(input$x, input$y, colour=input$color)) +
            geom_point(alpha=0.3)
        ##p
        print(p)
    })
}

ui <- fluidPage(

    headerPanel("My Data Explorer"),

    sidebarPanel(
        selectInput(inputId="y",
                    label="Y Variable",
                    choices=variables
                    ),
        selectInput(inputId="x",
                    label="X Variable",
                    choices=variables

                    ),
        selectInput(inputId="color",
                    label="Color by Variable",
                    choices=colors
                    )
    ),

    mainPanel(
        plotOutput("my_output_plot")
    )
    
)

In [ ]:
# deploy
# shinyApp(ui=ui, server=server)

### Quick notes
* 2 components: server and ui
* inputs in ui map to inputs in server function (and its arguments)
* note aes_string 
* highlight all instances of "input" and "output"

## Concepts

Here is a barebones app:

```library(shiny)

server <- function(input, output, session) { }

ui <- basicPage("This is a Shiny app with some text")

shinyApp(ui=ui, server=server) 
```

A shiny visualization has 2 components:

* **server** - serves up your plot
    * uses *reactive* functions like `renderPlot` which must wrap the user input
    * is a function of both input and output, since it receives the user inputs, and sends output back to ui
* **ui** - gets user input for use in server
    * contains the `*Panel` layout functions
    * obtains input from user, or specified defaults
    * styles the page

The link between these is the "input" and "output" code. A good exercise is to highlight these keywords in your shiny code.

Here is how this works:
1. Server: renderPlot() reads user input from UI (or default input the first
   time app is ran)
2. Server: renderPlot() is set as the output from server, and is passed to ui
3. UI: plotOutput shows the plot to the user

### Getting started

* Start with a page layout, such as `fluidPage`. Other common layouts are: `bootstrapPage`, `navbarPage`, or `fixedPage`. See http://shiny.rstudio.com/articles/layout-guide.html
* Decide what widgets you need. Start with a small number. See here: http://shiny.rstudio.com/gallery/widget-gallery.html
* Obtain user input via ui. Use the input id's, along with server's input
  argument. These are called "reactive values"
* You MUST wrap inputs in a reactive function: `observe`, `reactive`, `render*`

Reactive functions:
* observe: to update something; produce side effects; eg, print text debugging, update or update something

* reactive: isolate code and generate output, like from a function

### Final Notes
shiny converts R code into HTML, CSS, JS for web deployment. Both `server` and `ui` can now reside in the same file (previous versions of `shiny` required 2 separate files). And you no longer need `shinyUI`

## Takeaways

* be very careful with names! lowercase/uppercase, especially in `showOutput`
* also names in shiny have to be alphanumeric, no special characters!!
* input ids must be unique
* wrap all inputs with a reactive function: `observe`, `reactive`, `render*`
  [error: "operation not allowed without an active reactive context"]
* Using renderChart2 doesn't require the using `$set(dom = ....`


be careful what you put inside reactive functions:
* non-reactive inputs in reactive functions willonly run once
* reactive inputs in reactive functions will trigger the function, even when not used for anything

## References

[Official reference](http://shiny.rstudio.com/reference/shiny/latest/)

[Cheathseet!](https://www.rstudio.com/resources/cheatsheets/)

[Boilerplate code!](http://zevross.com/blog/2016/04/19/r-powered-web-applications-with-shiny-a-tutorial-and-cheat-sheet-with-40-example-apps/)

More:
* [Debugging in shiny](http://zevross.com/blog/2016/04/19/r-powered-web-applications-with-shiny-a-tutorial-and-cheat-sheet-with-40-example-apps/#before-going-any-further-let-me-introduce-the-update-functions)
* [Customization with themes](http://rstudio.github.io/shinythemes/)
* [You can do advanced styling with CSS, and also HTML tagging](http://zevross.com/blog/2016/04/19/r-powered-web-applications-with-shiny-a-tutorial-and-cheat-sheet-with-40-example-apps/#layout-your-user-interface)
* [Create a dynamic user interface](http://zevross.com/blog/2016/04/19/r-powered-web-applications-with-shiny-a-tutorial-and-cheat-sheet-with-40-example-apps/#dynamic-ui-with-renderui-and-outputui)
* [Push to shinyapps.io](http://shiny.rstudio.com/articles/shinyapps.html)
* [rCharts: another way to style](https://ramnathv.github.io/rCharts/)


## Exercise

Create an interactive bar chart for input dataset, by adapting the example plot

In [ ]:
## add a subset to the above scatterplot - filter by code
df0$DRGcode %<>% as.character

(codes <- df0$DRGcode %>% unique %>% sort)

In [ ]:
server <- function(input, output){
    
    dataset <- reactive({
        df0
        ## if (input$code != 'None')
        ##     df_filter <- df %>% filter(DRG.code==input$code)
        ## df_filter
    })

    output$my_output_plot <- renderPlot({

        p <- ggplot(dataset, aes_string(input$x, input$y, colour=input$color)) +
            geom_point(alpha=0.3)
        ##p
        print(p)

    })
}

In [ ]:
ui <- fluidPage(

    headerPanel("My Data Explorer"),

    sidebarPanel(
        selectInput(inputId="y", label="Y Variable", choices=variables),
        selectInput(inputId="x", label="X Variable", choices=variables),
        selectInput(inputId="color", label="Color by Variable", choices=colors),
        selectInput(inputId="code", label="Select DRG code", c('None', codes))
    ),

    mainPanel( plotOutput("my_output_plot") )
)

In [ ]:
# shinyApp(ui=ui, server=server)

*Copyright &copy; 2016 The Data Incubator.  All rights reserved.*